In [36]:
from ycharts_parsers import YchartsDataVar
import polars as pl
from glob import glob
from tqdm import tqdm
from os import listdir
import pandas as pd
import seaborn as sns

In [2]:
YCH_DATA_TYPE = YchartsDataVar.TOTAL_ASSETS
PARQUETS_PATH = f"./data/ycharts/{YCH_DATA_TYPE.value}_parqs"

In [3]:
len(listdir(PARQUETS_PATH))

2139

In [32]:
df_pl_res = [pl.read_parquet(fp).drop("index") for fp in tqdm(glob(f"{PARQUETS_PATH}/*.parquet"))]

100%|██████████| 2139/2139 [00:01<00:00, 2101.97it/s]


In [46]:
df_pl_res[10]

date,AAVVF
datetime[ns],f64
1984-12-01 00:00:00,1.5540e9
1985-12-01 00:00:00,2.0160e9
1987-03-01 00:00:00,2.0890e9
1988-03-01 00:00:00,1.9180e9
1989-03-01 00:00:00,1.8280e9
1990-03-01 00:00:00,1.5690e9
1991-03-01 00:00:00,1.0560e9
1992-03-01 00:00:00,1.1300e9
1993-03-01 00:00:00,1.5620e9


In [33]:
df_merged = df_pl_res[0]
for df in tqdm(df_pl_res[1:]):
    df_merged = df_merged.join(df, on="date", how="outer")
df_merged = df_merged.sort("date")
print(df_merged.shape)

100%|██████████| 2138/2138 [00:04<00:00, 497.47it/s]

(471, 2140)


In [42]:
df = df_merged.to_pandas().set_index("date")

In [53]:
df.resample("1Y").mean().loc["2010":].index

DatetimeIndex(['2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31',
               '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
               '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31',
               '2022-12-31', '2023-12-31'],
              dtype='datetime64[ns]', name='date', freq='A-DEC')

In [ ]:
df_merged.write_parquet("./ch_data/merged.parquet")
df_merged.write_csv("./ch_data/merged.csv")